---

## Environment Setup

In this section, we configure the necessary libraries and connect to the Groq API for natural language processing.

# Macro Risk AI Agent

## Project Overview
This notebook demonstrates an AI-powered SQL agent that analyses UK insolvency data (2000-2025) to identify macro-economic risk trends. The agent uses natural language queries to automatically generate and execute SQL commands, providing insights into corporate and individual financial distress patterns.

## Key Features
- **Natural Language SQL Generation**: Ask questions in plain English, get SQL-based answers
- **Multi-Table Analysis**: Joins corporate and individual insolvency data
- **Temporal Aggregation**: Automatically converts monthly data to yearly trends
- **Comparative Analytics**: Identifies trend reversals and ratio calculations

## Data Sources
- **Company Insolvencies**: [UK Insolvency Service - Company Insolvencies (November 2025)](https://www.gov.uk/government/statistics/company-insolvencies-november-2025)
- **Individual Insolvencies**: [UK Insolvency Service - Individual Insolvencies (November 2025)](https://www.gov.uk/government/statistics/individual-insolvencies-november-2025)

## Technical Stack
- **LLM**: Groq (llama-3.1-8b-instant)
- **Framework**: LangChain SQL Agent
- **Database**: SQLite
- **Data Processing**: Pandas

In [ ]:
# Import required libraries
import os  # Operating system interface
import pandas as pd  # Data manipulation and analysis
import sqlite3  # Lightweight SQL database
from langchain_groq import ChatGroq  # Groq LLM integration
from langchain_community.utilities import SQLDatabase  # LangChain SQL wrapper
from langchain_community.agent_toolkits import create_sql_agent  # Agent constructor
import time  # For rate limiting API calls
from dotenv import load_dotenv
import requests

# Configure API credentials (keep this secure!)
load_dotenv()
groq_key = os.getenv("GROQ_API_KEY")
if not groq_key:
    raise ValueError("GROQ_API_KEY not found. Please set it as an environment variable.")
os.environ["GROQ_API_KEY"] = groq_key

# database name and base GitHub URL for all data files
db_name = "macro_risk.db"
github_base_url = "https://raw.githubusercontent.com/Beck40/Gen-AI/main/data/"

---

## 1. Data Loading and Cleaning

This section loads the raw CSV files from the UK Insolvency Service and applies data quality transformations. The cleaning process handles government metadata codes and normalises column names for consistency.

In [14]:
# Data Cleaning Function
def clean_gov_data(df):
    """
    Cleans UK Gov data based on the Metadata file rules:
    - [z] = Not applicable (Convert to 0)
    - [x] = Not available (Convert to 0)
    - Removes commas from numbers (e.g. 1,000 -> 1000)
    """
    # Replace metadata codes with 0
    df = df.replace({'[z]': 0, '[x]': 0, ':': 0, '..': 0})
    
    # Ensure all numerical columns are actually numbers
    for col in df.columns:
        if col not in ['period', 'year', 'month']:
            if df[col].dtype == object:
                # Remove commas
                df[col] = df[col].astype(str).str.replace(',', '')
            # Force to numeric 
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

# LOAD INDIVIDUAL DATA
print("--- LOADING INDIVIDUAL DATA ---")
try:
    # Load CSV from GitHub
    df_ind = pd.read_csv(github_base_url + "Individual_Insolvency_November_2025.csv", header=0)
    
    # Map based on Metadata File
    rename_ind = {
        'year': 'year', 
        'month': 'month',
        'EW_total_individuals_SA': 'total_ind_insolvencies',
        'EW_bankruptcy_SA': 'bankruptcies',
        'EW_IVA_SA': 'ivas',
        'EW_DRO_SA': 'dros',
        'EW_self_employed_bankrupt_SA': 'trader_bankruptcies',  # Key SME Metric
        'EW_creditor_bankrupt_SA': 'forced_bankruptcies',       # Hostile
        'EW_debtor_bankrupt_SA': 'voluntary_bankruptcies'       # Strategic
    }
    
    df_ind = df_ind.rename(columns=rename_ind)
    # Only keep columns that actually exist in the file
    available_cols = [c for c in rename_ind.values() if c in df_ind.columns]
    df_ind = df_ind[available_cols]
    
    df_ind = clean_gov_data(df_ind)
    print(f"Loaded Individual Data: {len(df_ind)} rows.")

except Exception as e:
    print(f"Error loading Individual CSV: {e}")
    df_ind = pd.DataFrame()

# LOAD COMPANY DATA
print("--- LOADING COMPANY DATA ---")
try:
    # Load CSV from GitHub
    df_co = pd.read_csv(github_base_url + "Company_Insolvency_November_2025.csv", header=0)
    
    # Normalise headers (lowercase, no spaces) to match metadata keys
    df_co.columns = df_co.columns.str.strip().str.lower().str.replace(" ", "_")

    # Map based on Metadata File
    rename_co = {
        'year': 'year',
        'month': 'month',
        'total_ew_sa': 'total_co_insolvencies',
        'compliq_ew_sa': 'compulsory_liquidations', # Court Ordered
        'cvl_ew_sa': 'voluntary_liquidations',      # Director Led
        'admin_ew_sa': 'administrations'            # Rescue
    }
    
    df_co = df_co.rename(columns=rename_co)
    available_cols = [c for c in rename_co.values() if c in df_co.columns]
    df_co = df_co[available_cols]
    
    df_co = clean_gov_data(df_co)
    print(f"Loaded Company Data: {len(df_co)} rows.")

except Exception as e:
    print(f"Error loading Company CSV: {e}")
    df_co = pd.DataFrame()

--- LOADING INDIVIDUAL DATA ---
Loaded Individual Data: 311 rows.
--- LOADING COMPANY DATA ---
Loaded Company Data: 311 rows.


---

## 2. Database Creation

The cleaned data is loaded into a SQLite database. This creates two tables:
- **`retail_risk`**: Individual insolvencies (bankruptcies, IVAs, DROs, trader-specific metrics)
- **`corporate_risk`**: Company insolvencies (compulsory liquidations, voluntary liquidations, administrations)

In [ ]:
# --- LOAD DATABASE FROM GITHUB ---
try:
    r = requests.get(github_base_url + db_name)
    with open(db_name, "wb") as f:
        f.write(r.content)
    print(f"Downloaded {db_name} from GitHub to local directory")
except Exception as e:
    print(f"Error downloading {db_name}: {e}")

# --- DATABASE CREATION (Optional: Only needed if you want to re-create the database locally) ---
# print("--- DATABASE BUILT ---")
# conn = sqlite3.connect(db_name)
# df_ind.to_sql("retail_risk", conn, if_exists='replace', index=False)
# df_co.to_sql("corporate_risk", conn, if_exists='replace', index=False)
# conn.close()

Downloaded macro_risk.db from GitHub to local directory


---

## 3. AI Agent Configuration

This section initializes the LangChain SQL agent with a custom prompt that ensures accurate query generation. The agent is trained to:
- **Aggregate monthly data** into yearly totals using `SUM()` and `GROUP BY`
- **Join tables correctly** on both `year` AND `month` (critical for preventing data multiplication)
- **Handle different query types**: Trend analysis, comparisons, and ratio calculations
- **Output real data** without hallucination

The agent uses Groq's llama-3.1-8b-instant model for fast, cost-efficient natural language processing.

In [19]:
db = SQLDatabase.from_uri(f"sqlite:///{db_name}")
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

custom_table_info = """
Table: retail_risk
Columns: year, month, total_ind_insolvencies, trader_bankruptcies, forced_bankruptcies, voluntary_bankruptcies

Table: corporate_risk
Columns: year, month, total_co_insolvencies, compulsory_liquidations, voluntary_liquidations
"""

system_prefix = f"""
You are a Risk Analyst.
Schema:
{custom_table_info}

CRITICAL RULES:
1. AGGREGATION & SYNTAX:
   - ALWAYS use `SUM(column)` to aggregate monthly data into yearly totals.
   - CORRECT SQL ORDER: `SELECT ... FROM ... WHERE ... GROUP BY ...`
   - NEVER place `WHERE` after `GROUP BY`.

2. JOIN LOGIC (CRITICAL):
   - When querying both tables, you MUST join on BOTH 'year' AND 'month'.
   - Correct: `ON t1.year = t2.year AND t1.month = t2.month`
   - Wrong: `ON t1.year = t2.year` (This multiplies results by 12!)
   - If you see "ambiguous column name", use `table.column` (e.g., `retail_risk.year`).

3. QUESTION LOGIC:
   - "Calculate" / "Trend" (Single Table):
      - Query: `SELECT year, SUM(column) FROM table WHERE year IN (...) GROUP BY year`
   - "Ratio": 
      - Query: `SELECT SUM(t1.total_ind_insolvencies), SUM(t2.total_co_insolvencies) FROM retail_risk t1 JOIN corporate_risk t2 ON t1.year = t2.year AND t1.month = t2.month WHERE t1.year = 2023`
      - Do NOT group by month. Get the single yearly totals directly.
      - Calculate ratio in Final Answer.
   - "Compare" (YEAR-BY-YEAR):
      - Query MUST include: `SELECT t1.year, SUM(...), SUM(...) ... GROUP BY t1.year`
      - Example: `SELECT t1.year, SUM(t1.compulsory_liquidations), SUM(t2.forced_bankruptcies) FROM corporate_risk t1 JOIN retail_risk t2 ON t1.year = t2.year AND t1.month = t2.month WHERE t1.year BETWEEN 2015 AND 2023 GROUP BY t1.year`
      - CRITICAL: Do NOT omit `GROUP BY t1.year`. You need separate rows for each year.
      - After execution, compare each row individually.
      - For each tuple (year, corp_val, ind_val):
         - If corp_val > ind_val, say "Corp is higher"
         - If ind_val > corp_val, say "Ind is higher"
      - Format: "Year: Corp [Value] vs Ind [Value] -> [Winner] is higher"

4. EXECUTION:
   - You MUST execute the query and use the REAL numbers.
   - NEVER make up numbers or use placeholders like 12345.
   - If the query fails, fix it and try again.

5. TERMINATION:
   - Once you have the SQL observation, your ONLY valid next step is "Final Answer:".
   - Do NOT output "Action: Thought", "Action: None", or any other action.
   - Do NOT try to use python or any other tools.

6. FORMAT:
   - Final Answer must be based on the SQL result.
   - Do not output "Action:" after "Final Answer:".
"""

agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="zero-shot-react-description",
    prefix=system_prefix,
    verbose=False,
    handle_parsing_errors=True,
    max_iterations=8
)

---

## 4. Running the Analysis

The agent now answers three key macro-economic questions in plain English:

1. **Trend Analysis**: What were the yearly total Trader Bankruptcies for 2020-2023?
2. **Comparative Analysis**: How do Company Compulsory Liquidations compare to Individual Forced Bankruptcies (2015-2023)?
3. **Ratio Calculation**: What is the ratio of Individual to Corporate Insolvencies in 2023?

The agent will automatically generate SQL queries, execute them, and provide human-readable answers.

In [20]:
# Execute AI-Powered Analysis
print("\n--- MACRO RISK ANALYST ---")

# Define natural language questions for the agent
# The agent will convert these to SQL queries automatically
questions = [
    # Question 1: Trend analysis of self-employed bankruptcies (SME health indicator)
    "Calculate the yearly total Trader Bankruptcies for 2020, 2021, 2022, and 2023.",
    
    # Question 2: Comparative analysis showing trend reversal from consumer to business distress
    "Compare the yearly total Company Compulsory Liquidations vs Individual Forced Bankruptcies for the years 2015 through 2023.",
    
    # Question 3: Ratio calculation for macro-economic health indicator
    "For the year 2023, what is the ratio of Total Individual Insolvencies to Total Corporate Insolvencies?"
]

for q in questions:
    print(f"\nAnalyst: {q}")
    try:
        response = agent_executor.invoke(q)
        print(f"Agent: {response['output']}")
    except Exception as e:
        print(f"Error: {e}")
    
    time.sleep(5)


--- MACRO RISK ANALYST ---

Analyst: Calculate the yearly total Trader Bankruptcies for 2020, 2021, 2022, and 2023.
Agent: The yearly total Trader Bankruptcies for 2020, 2021, 2022, and 2023 are 2234, 1402, 1227, and 1493 respectively.

Analyst: Compare the yearly total Company Compulsory Liquidations vs Individual Forced Bankruptcies for the years 2015 through 2023.
Agent: 2015: Corp 2853 vs Ind 4404 -> Ind is higher
2016: Corp 2885 vs Ind 3901 -> Ind is higher
2017: Corp 2747 vs Ind 3305 -> Ind is higher
2018: Corp 3090 vs Ind 3661 -> Ind is higher
2019: Corp 2941 vs Ind 3128 -> Ind is higher
2020: Corp 1354 vs Ind 1347 -> Corp is higher
2021: Corp 492 vs Ind 1113 -> Ind is higher
2022: Corp 1968 vs Ind 1143 -> Corp is higher
2023: Corp 2838 vs Ind 1697 -> Corp is higher

Analyst: For the year 2023, what is the ratio of Total Individual Insolvencies to Total Corporate Insolvencies?
Agent: The ratio of Total Individual Insolvencies to Total Corporate Insolvencies for the year 2023 is

---

## 5. Ground Truth Verification

To validate the AI agent's accuracy, this section calculates the correct answers using Pandas and direct SQL queries. This provides:
- **Quality assurance**: Ensures the agent is generating correct SQL and interpreting results accurately
- **Debugging support**: Identifies discrepancies between agent output and expected results
- **Transparency**: Shows the ground truth data for independent verification

In [21]:
# Verification: Calculate Ground Truth Answers
# This section uses Pandas to independently verify the AI agent's SQL queries
print("--- GROUND TRUTH VERIFICATION ---")

# Q1: Trader Bankruptcies 2020-2023
q1_ans = df_ind[df_ind['year'].isin([2020, 2021, 2022, 2023])].groupby('year')['trader_bankruptcies'].sum()
print(f"Q1 Correct Answers (Pandas):\n{q1_ans}")

# Verify SQL directly to ensure it is populated correctly
print("\nQ1 SQL Check (Direct DB Query):")
try:
    sql_check = db.run("SELECT year, SUM(trader_bankruptcies) FROM retail_risk WHERE year IN (2020, 2021, 2022, 2023) GROUP BY year")
    print(sql_check)
except Exception as e:
    print(f"SQL Check Failed: {e}")

# Q2: Compulsory Liquidations vs Forced Bankruptcies 2015-2023
years_q2 = range(2015, 2024)
q2_co = df_co[df_co['year'].isin(years_q2)].groupby('year')['compulsory_liquidations'].sum()
q2_ind = df_ind[df_ind['year'].isin(years_q2)].groupby('year')['forced_bankruptcies'].sum()
q2_df = pd.DataFrame({'Compulsory Liq': q2_co, 'Forced Bankruptcies': q2_ind})
print(f"\nQ2 Correct Answers:\n{q2_df}")

# Q3: Ratio 2023
ind_2023 = df_ind[df_ind['year'] == 2023]['total_ind_insolvencies'].sum()
co_2023 = df_co[df_co['year'] == 2023]['total_co_insolvencies'].sum()
ratio = ind_2023 / co_2023
print(f"\nQ3 Correct Answer: Ind: {ind_2023}, Corp: {co_2023}, Ratio: {ratio:.2f}")

--- GROUND TRUTH VERIFICATION ---
Q1 Correct Answers (Pandas):
year
2020    2234
2021    1402
2022    1227
2023    1493
Name: trader_bankruptcies, dtype: int64

Q1 SQL Check (Direct DB Query):
[(2020, 2234), (2021, 1402), (2022, 1227), (2023, 1493)]

Q2 Correct Answers:
      Compulsory Liq  Forced Bankruptcies
year                                     
2015            2853                 4404
2016            2885                 3901
2017            2747                 3305
2018            3090                 3661
2019            2941                 3128
2020            1354                 1347
2021             492                 1113
2022            1968                 1143
2023            2838                 1697

Q3 Correct Answer: Ind: 103434, Corp: 25162, Ratio: 4.11


---

## Key Insights

From the analysis above, we can observe:

1. **Trader Bankruptcies (SME Metric)**: The COVID-19 pandemic (2020-2021) showed a significant drop in trader bankruptcies due to government support schemes, followed by a sharp recovery in 2022-2023.

2. **Compulsory Liquidations vs. Forced Bankruptcies**: 
   - **2015-2019**: Individual forced bankruptcies were consistently higher, reflecting consumer-driven distress.
   - **2020**: A trend reversal began (Corp: 1,354 vs Ind: 1,347).
   - **2022-2023**: Corporate compulsory liquidations surged, indicating increased business distress post-pandemic.

3. **2023 Insolvency Ratio**: The Individual-to-Corporate insolvency ratio provides a macro-economic health indicator. A higher ratio suggests consumer stress; a lower ratio indicates business sector challenges.

---

## Technical Challenges Solved

1. **Data Multiplication Bug**: Initially, queries joined on `year` only, causing 12x inflation (one row per month). Fixed by joining on both `year` AND `month`.
2. **Agent Hallucination**: The LLM attempted to calculate comparisons without executing queries. Resolved by enforcing strict prompt rules: "Execute SQL first, then output Final Answer."
3. **SQL Syntax Errors**: The agent placed `WHERE` after `GROUP BY`. Fixed by explicitly stating: "Correct SQL order: SELECT ... FROM ... WHERE ... GROUP BY".
4. **Infinite Loops**: The agent tried to use `Action: None` or `Action: Thought`, causing parsing errors. Prevented by adding termination rules to the system prompt.
---